# 飞桨常规赛：遥感影像地块分割 - 12月第6名方案
## 赛题任务
本赛题旨在对遥感影像进行像素级内容解析，并对遥感影像中感兴趣的类别进行提取和分类，以衡量遥感影像地块分割模型在多个类别（如建筑、道路、林地等）上的效果。

## 数据说明
本赛题提供了多个地区已脱敏的遥感影像数据，各参赛选手可以基于这些数据构建自己的地块分割模型。

## 训练数据集
样例图片及其标注如下图所示：
![](https://ai-studio-static-online.cdn.bcebos.com/61920df9736a47f1b3d7764e28720bb199ac0b07caa84185a1f96a2f426d710b)

训练数据集文件名称：train_and_label.zip
包含2个子文件，分别为：训练数据集（原始图片）文件、训练数据集（标注图片）文件，详细介绍如下：

    训练数据集（原始图片）文件名称：img_train
    包含66,653张分辨率为2m/pixel，尺寸为256 * 256的JPG图片，每张图片的名称形如T000123.jpg。

    训练数据集（标注图片）文件名称：lab_train
    包含66,653张分辨率为2m/pixel，尺寸为256 * 256的PNG图片，每张图片的名称形如T000123.png。
    备注： 全部PNG图片共包括4种分类，像素值分别为0、1、2、3。此外，像素值255为未标注区域，表示对应区域的所属类别并不确定，在评测中也不会考虑这部分区域。

测试数据集

测试数据集文件名称：img_test.zip，详细介绍如下：
包含4,609张分辨率为2m/pixel，尺寸为256 * 256的JPG图片，文件名称形如123.jpg。


# 下载PaddleSeg套件

In [ ]:
# 下载paddlesegv2.1版本
!git clone -b release/2.1 https://gitee.com/paddlepaddle/PaddleSeg.git

# 一、数据准备

## 1.1 解压数据

In [1]:
!unzip -oq /home/aistudio/data/data80164/train_and_label.zip

In [2]:
!unzip -oq /home/aistudio/data/data80164/img_test.zip

In [4]:
%cd ~

/home/aistudio


## 1.2 对遥感地块分割的训练集、划分训练集、验证集

In [5]:

import random
from pathlib import Path

root_path = Path(r'/home/aistudio')

images_path = root_path/'img_train'
labels_path = root_path/'lab_train'

train_txt = open(root_path/'train.txt', 'w')
val_txt = open(root_path/'val.txt', 'w')

for label_path in labels_path.rglob('*.png'):
    if random.random() < 0.9:
        train_txt.write(f'{images_path}/{label_path.stem}.jpg {labels_path}/{label_path.name} \n')
    else:
        val_txt.write(f'{images_path}/{label_path.stem}.jpg {labels_path}/{label_path.name} \n')

train_txt.close()
val_txt.close()

# 二、训练准备

## 2.1 复制训练配置文件和基准文件

In [6]:
!cp -r ccf_bci.yml  PaddleSeg/configs/_base_/ 
!mkdir PaddleSeg/configs/ccf_bci
!cp -r model.yml  PaddleSeg/configs/ccf_bci/ 

```# _base_/ccf_bci.yml


# 存放对数据集以及优化器的操作

train_dataset:
  type: Dataset
  dataset_root: /home/aistudio/
  train_path: /home/aistudio/train.txt
  num_classes: 4
  transforms:
    - type: ResizeStepScaling
      min_scale_factor: 0.5
      max_scale_factor: 2.0
      scale_step_size: 0.25
    - type: RandomPaddingCrop
      crop_size: [256, 256]
    - type: RandomHorizontalFlip
    - type: RandomDistort
      brightness_range: 0.4
      contrast_range: 0.4
      saturation_range: 0.4
    - type: Normalize
  mode: train

val_dataset:
  type: Dataset
  dataset_root: /home/aistudio/
  val_path: /home/aistudio/val.txt
  num_classes: 4
  transforms:
    - type: Normalize
  mode: val


optimizer:
  type: sgd
  momentum: 0.9
  weight_decay: 4.0e-5

lr_scheduler:
  type: PolynomialDecay
  learning_rate: 0.001
  end_lr: 0
  power: 0.9
```

```# model.yml
_base_: '../_base_/ccf_bci.yml'

batch_size: 64
iters: 80000


optimizer:
  type: sgd
  momentum: 0.9
  weight_decay: 4.0e-5

lr_scheduler:
  type: PolynomialDecay
  learning_rate: 0.01
  end_lr: 0
  power: 0.9

loss:
  types:
    - type: MixedLoss
      losses:
        - type: CrossEntropyLoss
        - type: LovaszSoftmaxLoss
        - type: DiceLoss
      coef: [0.6, 0.1, 0.3]
  coef: [1]

model:
  type: FCN
  backbone:
    type: HRNet_W48
    align_corners: False
    pretrained: https://bj.bcebos.com/paddleseg/dygraph/hrnet_w48_ssld.tar.gz
  backbone_indices: [-1]
  pretrained: Null
```

## 2.3 开始训练

In [7]:
# 项目运行
%cd ~

# 模型训练

!python PaddleSeg/train.py \
       --config PaddleSeg/configs/ccf_bci/model.yml \
       --do_eval \
       --use_vdl \
       --save_interval 9360 \
       --save_dir output_loss_mixed_1 \

/home/aistudio
/home/aistudio/PaddleSeg/paddleseg/cvlibs/param_init.py:89: DeprecationWarning: invalid escape sequence \s
  """
/home/aistudio/PaddleSeg/paddleseg/models/losses/binary_cross_entropy_loss.py:82: DeprecationWarning: invalid escape sequence \|
  """
/home/aistudio/PaddleSeg/paddleseg/models/losses/lovasz_loss.py:50: DeprecationWarning: invalid escape sequence \i
  """
/home/aistudio/PaddleSeg/paddleseg/models/losses/lovasz_loss.py:77: DeprecationWarning: invalid escape sequence \i
  """
/home/aistudio/PaddleSeg/paddleseg/models/losses/lovasz_loss.py:120: DeprecationWarning: invalid escape sequence \i
  """
2022-01-19 14:30:22 [INFO]	
------------Environment Information-------------
platform: Linux-4.4.0-150-generic-x86_64-with-debian-stretch-sid
Python: 3.7.4 (default, Aug 13 2019, 20:35:49) [GCC 7.3.0]
Paddle compiled with cuda: True
NVCC: Cuda compilation tools, release 10.1, V10.1.243
cudnn: 7.6
GPUs used: 1
CUDA_VISIBLE_DEVICES: None
GPU: ['GPU 0: Tesla V100-SXM2-32GB'

模型预测时希望将预测的结果直接作为提交结果，但是paddleseg默认预测的结果是增加权重后生成的图片，所以对paddleseg的源码进行了修改。修改的文件为work/PaddleSeg/paddleseg/core/predict.py，修改后是下面这段代码

In [8]:
!cp -r predict.py  PaddleSeg/paddleseg/core/ 

```
# Copyright (c) 2020 PaddlePaddle Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#    http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.


import os
import math

import cv2
import numpy as np
import paddle

from paddleseg import utils
from paddleseg.core import infer
from paddleseg.utils import logger, progbar


def mkdir(path):
    sub_dir = os.path.dirname(path)
    if not os.path.exists(sub_dir):
        os.makedirs(sub_dir)


def partition_list(arr, m):
    """split the list 'arr' into m pieces"""
    n = int(math.ceil(len(arr) / float(m)))
    return [arr[i:i + n] for i in range(0, len(arr), n)]


# 修改predict函数
def predict(model,
            model_path,
            transforms,
            image_list,
            image_dir=None,
            save_dir='output',
            aug_pred=False,
            scales=1.0,
            flip_horizontal=True,
            flip_vertical=False,
            is_slide=False,
            stride=None,
            crop_size=None):
    """
    predict and visualize the image_list.

    Args:
        model (nn.Layer): Used to predict for input image.
        model_path (str): The path of pretrained model.
        transforms (transform.Compose): Preprocess for input image.
        image_list (list): A list of image path to be predicted.
        image_dir (str, optional): The root directory of the images predicted. Default: None.
        save_dir (str, optional): The directory to save the visualized results. Default: 'output'.
        aug_pred (bool, optional): Whether to use mulit-scales and flip augment for predition. Default: False.
        scales (list|float, optional): Scales for augment. It is valid when `aug_pred` is True. Default: 1.0.
        flip_horizontal (bool, optional): Whether to use flip horizontally augment. It is valid when `aug_pred` is True. Default: True.
        flip_vertical (bool, optional): Whether to use flip vertically augment. It is valid when `aug_pred` is True. Default: False.
        is_slide (bool, optional): Whether to predict by sliding window. Default: False.
        stride (tuple|list, optional): The stride of sliding window, the first is width and the second is height.
            It should be provided when `is_slide` is True.
        crop_size (tuple|list, optional):  The crop size of sliding window, the first is width and the second is height.
            It should be provided when `is_slide` is True.

    """
    utils.utils.load_entire_model(model, model_path)
    model.eval()
    nranks = paddle.distributed.get_world_size()
    local_rank = paddle.distributed.get_rank()
    if nranks > 1:
        img_lists = partition_list(image_list, nranks)
    else:
        img_lists = [image_list]

    added_saved_dir = os.path.join(save_dir, 'added_prediction')
    pred_saved_dir = os.path.join(save_dir, 'pseudo_color_prediction')

    logger.info("Start to predict...")
    progbar_pred = progbar.Progbar(target=len(img_lists[0]), verbose=1)
    with paddle.no_grad():
        for i, im_path in enumerate(img_lists[local_rank]):
            im = cv2.imread(im_path)
            ori_shape = im.shape[:2]
            im, _ = transforms(im)
            im = im[np.newaxis, ...]
            im = paddle.to_tensor(im)

            if aug_pred:
                pred = infer.aug_inference(
                    model,
                    im,
                    ori_shape=ori_shape,
                    transforms=transforms.transforms,
                    scales=scales,
                    flip_horizontal=flip_horizontal,
                    flip_vertical=flip_vertical,
                    is_slide=is_slide,
                    stride=stride,
                    crop_size=crop_size)
            else:
                pred = infer.inference(
                    model,
                    im,
                    ori_shape=ori_shape,
                    transforms=transforms.transforms,
                    is_slide=is_slide,
                    stride=stride,
                    crop_size=crop_size)
            pred = paddle.squeeze(pred)
            pred = pred.numpy().astype('uint8')

            # get the saved name
            if image_dir is not None:
                im_file = im_path.replace(image_dir, '')
            else:
                im_file = os.path.basename(im_path)
            if im_file[0] == '/' or im_file[0] == '\\':
                im_file = im_file[1:]

            # 修改预测后的图片
            pred_saved_path = os.path.join(save_dir, im_file.rsplit(".")[0] + ".png")
            mkdir(pred_saved_path)
            cv2.imwrite(pred_saved_path, pred)

            progbar_pred.update(i + 1)
            ```


# 三、预测
将预测完成后的文件提取打包提交即可

In [ ]:
%cd ~
# 模型预测
!python PaddleSeg/predict.py \
       --config PaddleSeg/configs/ccf_bci/model.yml \
       --model_path output_loss_mixed_1/best_model/model.pdparams \
       --image_path img_testA \
       --save_dir result/